In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from typing import List

In [3]:
def parseMe(url: str,headers: str) -> BeautifulSoup:
    response =requests.get(url,headers=headers)
    #print(response)
    assert response.status_code==200, "Page didn't respond"
    pg = response.content
    return BeautifulSoup(pg,'html.parser')


In [4]:
def scrapeMe(page: BeautifulSoup) -> List[str]:
    a = page.find('div','hotel-facilities__list').find_all('div',class_='bui-spacer--large')

    types=[]
    avails=[]

    for i in a:
        types.append(i.find('div', class_="bui-title__text hotel-facilities-group__title-text").get_text().split('\n')[1])
        avail = i.find('ul').find_all('li')

        s=""
        for j in avail:
            if j!=avail[-1]:
                s +=  j.find('div', class_="bui-list__body").get_text().split('\n\n')[1] + ' | '
            else:
                s +=  j.find('div', class_="bui-list__body").get_text().split('\n\n')[1]
        avails.append(s)
        
    return types,avails


In [5]:
def dictMe(types: List[str], avails: List[str]) -> dict:
    assert len(types)==len(avails), "The lengths don't match"
    d=dict(zip(types,avails))
    return d


In [6]:
def pdMe(d: dict) -> pd.DataFrame:
    df = pd.DataFrame([d])
    return df


In [7]:
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
url ='https://www.booking.com/hotel/am/bed-and-breakfast-dilijan-orran.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaAeIAQGYATG4ARnIAQzYAQHoAQH4AQKIAgGoAgO4At-Qs5gGwAIB0gIkNjYzZDQ0Y2YtMzMyZi00YjJiLWI1ODYtOTUzOTNhMGEwYTYz2AIF4AIB&sid=4c9da70ca7c98f7e6414f416852ae238&dest_id=-2324702;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=2;hpos=2;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1661782167;srpvid=1e91638b6fd3024d;type=total;ucfs=1&#hp_facilities_box'

def scMeBooking(url: str,headers: str) -> pd.DataFrame:
    pg = parseMe(url, headers)
    a,b = scrapeMe(pg)
    d = dictMe(a,b)
    return pdMe(d)
    

df=scMeBooking(url,headers)